Install dependencies

In [ ]:
! pip install -q transformers datasets torch nervaluate seqeval scikit-learn pandas tqdm evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.9 MB/s eta 0:00:00


In [ ]:
import os
# Set environment variable to help mitigate CUDA memory fragmentation.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# HF_TOKEN_PLACEHOLDER

Load and Preprocess the Dataset

In [ ]:
import pandas as pd
import numpy as np
import ast
import json
import torch
import evaluate
import random
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModel,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification,
    set_seed
)
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
set_seed(70)
random.seed(70)
np.random.seed(70)
torch.manual_seed(70)

In [ ]:
# Load the datasets
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

In [ ]:
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

print("\nTrain columns:", train_df.columns.tolist())
print("\nMissing values in train:\n", train_df.isnull().sum())

# Peek into how entities look
print("\nExample entities format:\n", train_df['entities'].iloc[0])

Train shape: (677, 3)
Test shape: (170, 2)

Train columns: ['text', 'id', 'entities']

Missing values in train:
 text        0
id          0
entities    0
dtype: int64

Example entities format:
 [{'end': 68, 'label': 'PER', 'start': 36}
 {'end': 86, 'label': 'LOC', 'start': 74}
 {'end': 177, 'label': 'PER', 'start': 122}
 {'end': 192, 'label': 'LOC', 'start': 184}
 {'end': 325, 'label': 'LOC', 'start': 311}
 {'end': 501, 'label': 'MSR-OTH', 'start': 488}]


In [ ]:
import re

def fix_and_parse_entities(s):
    if isinstance(s, list):
        return s  # Already parsed

    try:
        # Insert commas between closing and opening braces
        fixed = re.sub(r'\}(\s*)\{', r'}, {', s.strip())

        # If it already starts and ends with brackets, don't slice them off
        if fixed.startswith('[') and fixed.endswith(']'):
            return ast.literal_eval(fixed)
        else:
            return ast.literal_eval(f"[{fixed}]")

    except Exception as e:
        print("Fixing failed on:", s)
        print("Error:", e)
        return []

train_df['entities'] = train_df['entities'].apply(fix_and_parse_entities)

In [ ]:
print(train_df.head())

                                                text     id  \
0  SS : 158 \n In stämnde målet emellan Bonden An...  72535   
1  Grefwe von Schwerin F. B. anſåg de ifrågawaran...  23682   
2  Torg= och Hamnpriſer i Stockholm d. 7 Maj . Ha...  72477   
3  Hufvudstaden . \n Frälsningsarméns välgörenhet...  72715   
4  Hõgwãllborne Lands Hõfdinge samt Riddare af Ko...  24330   

                                            entities  
0  [{'end': 68, 'label': 'PER', 'start': 36}, {'e...  
1  [{'end': 25, 'label': 'PER', 'start': 0}, {'en...  
2  [{'end': 32, 'label': 'LOC', 'start': 23}, {'e...  
3  [{'end': 33, 'label': 'ORG-INST', 'start': 17}...  
4  [{'end': 27, 'label': 'OCC', 'start': 13}, {'e...  


In [ ]:
print(type(train_df['entities'].iloc[0]))        # should be <class 'list'>
print(type(train_df['entities'].iloc[0][0]))     # should be <class 'dict'>
print(train_df['entities'].iloc[0])

<class 'list'>
<class 'dict'>
[{'end': 68, 'label': 'PER', 'start': 36}, {'end': 86, 'label': 'LOC', 'start': 74}, {'end': 177, 'label': 'PER', 'start': 122}, {'end': 192, 'label': 'LOC', 'start': 184}, {'end': 325, 'label': 'LOC', 'start': 311}, {'end': 501, 'label': 'MSR-OTH', 'start': 488}]


Split the data (train and eval)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

entity_types = [
    'EVN', 'LOC', 'MSR-AREA', 'MSR-DIST', 'MSR-LEN', 'MSR-MON', 'MSR-OTH',
    'MSR-VOL', 'MSR-WEI', 'OCC', 'ORG-COMP', 'ORG-INST', 'ORG-OTH', 'PER',
    'SYMP', 'TME-DATE', 'TME-INTRV', 'TME-TIME', 'WRK'
]

# Extract set of labels for each text
def extract_labels(entities):
    return list(set(e['label'] for e in entities))

train_df['label_set'] = train_df['entities'].apply(extract_labels)

# Convert to binary matrix
mlb = MultiLabelBinarizer(classes=entity_types)
y = mlb.fit_transform(train_df['label_set'])

In [ ]:
!pip install scikit-multilearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
from skmultilearn.model_selection import iterative_train_test_split

X = train_df.values  # need this as NumPy array for skmultilearn
X_train, y_train, X_val, y_val = iterative_train_test_split(X, y, test_size=0.15)

# Convert back to DataFrame
train_df_split = pd.DataFrame(X_train, columns=train_df.columns)
val_df_split = pd.DataFrame(X_val, columns=train_df.columns)

In [ ]:
# Drop helper label columns
train_df_split = train_df_split.drop(columns=['label_set'])
val_df_split = val_df_split.drop(columns=['label_set'])

# Reset index
train_df_split.reset_index(drop=True, inplace=True)
val_df_split.reset_index(drop=True, inplace=True)

In [ ]:
print("Train size:", len(train_df_split))
print("Validation size:", len(val_df_split))
print(train_df_split.head())
print(val_df_split.head())


Train size: 568
Validation size: 109
                                                text     id  \
0  Grefwe von Schwerin F. B. anſåg de ifrågawaran...  23682   
1  Hufvudstaden . \n Frälsningsarméns välgörenhet...  72715   
2  Sammanträden . \n Drätſel = Kommisſionen ſamma...  38238   
3  Nott i bokhandelu . \n Männiſkokroppen , deß b...  72700   
4  Blandade Ämnen . \n Som bewis på priſerna i Ca...  72475   

                                            entities  
0  [{'end': 25, 'label': 'PER', 'start': 0}, {'en...  
1  [{'end': 33, 'label': 'ORG-INST', 'start': 17}...  
2  [{'end': 39, 'label': 'ORG-INST', 'start': 17}...  
3  [{'end': 58, 'label': 'WRK', 'start': 22}, {'e...  
4  [{'end': 54, 'label': 'LOC', 'start': 43}, {'e...  
                                                text     id  \
0  SS : 158 \n In stämnde målet emellan Bonden An...  72535   
1  Torg= och Hamnpriſer i Stockholm d. 7 Maj . Ha...  72477   
2  Hõgwãllborne Lands Hõfdinge samt Riddare af Ko...  24330   
3 

Define the full label list and mappings

In [ ]:
# Create BIO-formatted labels
unique_labels = ['O']
for label in entity_types:
    unique_labels.extend([f'B-{label}', f'I-{label}'])

label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

Load tokenizer

In [ ]:
#model_checkpoint = "bert-base-multilingual-cased"
#tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model_checkpoint = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Create a character-level label vector for each row

In [ ]:
def create_char_labels(text, entities):
    char_labels = ['O'] * len(text)
    for ent in entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        if end <= len(text):
            char_labels[start] = f'B-{label}'
            for i in range(start + 1, end):
                char_labels[i] = f'I-{label}'
    return char_labels

Tokenize and align labels with tokens

In [ ]:
def tokenize_and_align_labels(examples):
    all_input_ids = []
    all_attention_masks = []
    all_labels = []

    for text, entities in zip(examples['text'], examples['entities']):
        # Create character-level label array
        char_labels = ['O'] * len(text)
        for ent in entities:
            start, end, label = ent['start'], ent['end'], ent['label']
            if end <= len(text):
                char_labels[start] = f'B-{label}'
                for i in range(start + 1, end):
                    char_labels[i] = f'I-{label}'

        # Tokenize with overflowing chunks
        tokenized = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_offsets_mapping=True,
            return_overflowing_tokens=True,
            stride=128
        )

        for i in range(len(tokenized['input_ids'])):
            input_ids = tokenized['input_ids'][i]
            attention_mask = tokenized['attention_mask'][i]
            offsets = tokenized['offset_mapping'][i]

            labels = []
            for start_char, end_char in offsets:
                if start_char == end_char:
                    labels.append(-100)
                elif start_char < len(char_labels):
                    labels.append(label2id.get(char_labels[start_char], 0))
                else:
                    labels.append(0)  # fallback to 'O'

            all_input_ids.append(input_ids)
            all_attention_masks.append(attention_mask)
            all_labels.append(labels)

    return {
        "input_ids": all_input_ids,
        "attention_mask": all_attention_masks,
        "labels": all_labels
    }

In [ ]:
train_dataset = Dataset.from_pandas(train_df_split)
val_dataset = Dataset.from_pandas(val_df_split)

tokenized_train = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_val = val_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=val_dataset.column_names
)

Map:   0%|          | 0/568 [00:00<?, ? examples/s]

Map:   0%|          | 0/109 [00:00<?, ? examples/s]

In [ ]:
i = 0
example = train_df_split.iloc[i]
print("Tokens:", tokenizer.tokenize(example['text']))
print("Entities:", example['entities'])
print("Labels:", tokenized_train[i]['labels'])


Token indices sequence length is longer than the specified maximum sequence length for this model (632 > 512). Running this sequence through the model will result in indexing errors


Tokens: ['▁Gre', 'f', 'we', '▁von', '▁Schwer', 'in', '▁F', '.', '▁B', '.', '▁an', 'såg', '▁de', '▁i', 'fråga', 'waran', 'de', '▁skatt', 'erne', '▁', ',', '▁bland', '▁alla', '▁dem', '▁som', '▁Swe', 'nska', '▁Folke', 't', '▁är', 'ft', '▁af', '▁sina', '▁f', 'äder', '▁', ',', '▁war', 'a', '▁när', 'mast', '▁i', '▁', 'ordningen', '▁att', '▁ut', 'gå', '▁', '.', '▁De', '▁inne', 'bur', 'e', '▁i', '▁', 'sje', 'lf', 'wa', '▁sin', '▁historia', '▁ett', '▁frö', '▁till', '▁sin', '▁upp', 'lösning', '▁', '.', '▁Man', 'tals', 'pen', 'ningar', 'ne', '▁hade', '▁i', '▁må', 'n', '▁af', '▁my', 'nt', 'förändring', 'ar', '▁', ',', '▁ned', 'gått', '▁från', '▁8', '▁ka', 'ppar', '▁sä', 'd', '▁till', '▁12', '▁', 'ß', '▁', '.', '▁De', '▁wo', 're', '▁så', '▁o', 'jem', 'nt', '▁fördel', 'ade', '▁', ',', '▁att', '▁i', '▁wi', 'ß', 'a', '▁or', 'ter', '▁betala', 'de', '▁Tor', 'par', 'en', '▁i', '▁det', '▁en', 'a', '▁Här', 'a', 'det', '▁12', '▁', 'ß', '▁', '.', '▁då', '▁i', '▁det', '▁andra', '▁Hem', 'man', 'se', 'gare', '▁

Model Definition

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    #"bert-base-multilingual-cased",
    "xlm-roberta-large",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Arguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner-output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    #report_to="None",
    logging_steps=50,
    learning_rate=4.126064415780025e-05,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.060508612481530805,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Compute metrics

In [ ]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_preds = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

<ipython-input-26-16d1f445b7a3>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#847ea4b8ddc1a38b87a7a2d6f52da4c0fdb73df5

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
#import wandb
#wandb.init(project="ner-xlm-roberta")

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bwahle-ieu2021 (bwahle-ieu2021-ie-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.138500,0.211466,0.566530,0.709441,0.629983,0.948897
2,0.108100,0.170173,0.674378,0.753842,0.711900,0.960848
3,0.114900,0.201569,0.659626,0.731398,0.693660,0.955756
4,0.067200,0.210362,0.716006,0.754086,0.734553,0.961377
5,0.044500,0.226395,0.693882,0.769212,0.729608,0.960988
6,0.030900,0.249745,0.696916,0.766284,0.729956,0.961564
7,0.032600,0.253930,0.679487,0.749939,0.712977,0.959143
8,0.019300,0.284149,0.699772,0.747743,0.722963,0.959688
9,0.013500,0.284675,0.719726,0.769944,0.743989,0.961167
10,0.011700,0.303263,0.730501,0.767748,0.748662,0.962973


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are i

TrainOutput(global_step=9160, training_loss=0.03418530084692983, metrics={'train_runtime': 7714.0044, 'train_samples_per_second': 4.747, 'train_steps_per_second': 1.187, 'total_flos': 3.401351648679936e+16, 'train_loss': 0.03418530084692983, 'epoch': 20.0})

In [ ]:
test_df = pd.read_csv("test.csv")
test_dataset = Dataset.from_pandas(test_df)

def tokenize_test(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        stride=128
    )

tokenized_test = test_dataset.map(
    tokenize_test,
    batched=True,
    remove_columns=test_dataset.column_names
)


Map:   0%|          | 0/170 [00:00<?, ? examples/s]

In [ ]:
predictions = trainer.predict(tokenized_test)
preds = np.argmax(predictions.predictions, axis=2)

In [ ]:
def decode_predictions(tokenized_dataset, preds):
    all_entities = {}

    for i, example in enumerate(tokenized_dataset):
        input_ids = example['input_ids']
        offset_mapping = example['offset_mapping']
        pred = preds[i]

        # Get the ID of the original test row (we use overflowing tokens, so this may be repeated)
        orig_idx = example['overflow_to_sample_mapping'] if 'overflow_to_sample_mapping' in example else i
        text = test_df.iloc[orig_idx]['text']
        id_ = test_df.iloc[orig_idx]['id']

        if id_ not in all_entities:
            all_entities[id_] = []

        current_entity = None

        for idx, label_id in enumerate(pred):
            label = id2label[label_id]
            offset = offset_mapping[idx]

            if offset[0] == offset[1]:  # skip special tokens
                continue

            start_char, end_char = offset

            if label.startswith("B-"):
                if current_entity:
                    all_entities[id_].append(current_entity)
                current_entity = {
                    "label": label[2:],
                    "start": start_char,
                    "end": end_char
                }
            elif label.startswith("I-") and current_entity and label[2:] == current_entity["label"]:
                current_entity["end"] = end_char
            else:
                if current_entity:
                    all_entities[id_].append(current_entity)
                    current_entity = None

        if current_entity:
            all_entities[id_].append(current_entity)

    return all_entities

In [ ]:
decoded_entities = decode_predictions(tokenized_test, preds)

In [ ]:
submission = pd.DataFrame({
    "id": test_df["id"],
    "entities": test_df["id"].apply(lambda id_: json.dumps(decoded_entities.get(id_, [])))
})
submission.to_csv("submission.csv", index=False)
